In [17]:
from scipy.stats import ttest_rel, ttest_ind, wilcoxon

In [18]:
# Sample data - RMSE scores for original and replica models
pls_original = [3.39, 1.33, 2.97, 2.71, 1.28, 0.35, 1.09, 0.62]
pls_replica = [11.91, 0.81, 3.72, 7.26, 3.13, 3.58, 3.02, 1.40]

ica_original = [8.31, 1.44, 4.77, 5.17, 4.08, 3.07, 2.29, 0.98]
ica_replica = [9.89, 0.55, 4.25, 3.74, 3.13, 3.58, 3.02, 1.40]

moc_original = [5.30, 1.03, 3.47, 2.31, 2.21, 2.72, 0.62, 0.82]
moc_replica = [6.97, 0.52, 3.66, 3.45, 1.75, 5.75, 2.38, 0.91]


In [19]:
def perform_student_t_test(original, replica):
	t_stat, p_value = ttest_rel(original, replica)
	return t_stat, p_value

print(f"PLS Student's t-test: {perform_student_t_test(pls_original, pls_replica)}")
print(f"ICA Student's t-test: {perform_student_t_test(ica_original, ica_replica)}")
print(f"MOC Student's t-test: {perform_student_t_test(moc_original, moc_replica)}")


PLS Student's t-test: (-2.4668003444460473, 0.04303427458583176)
ICA Student's t-test: (0.18864352947756194, 0.8557248260240399)
MOC Student's t-test: (-1.9565114305012332, 0.0912876922693679)


In [20]:
def perform_welchs_test(original, replica):
    t_stat, p_value = ttest_ind(original, replica, equal_var=False)
    return t_stat, p_value

print(f"PLS1-SM Welch's: {perform_welchs_test(pls_original, pls_replica)}")
print(f"ICA Welch's: {perform_welchs_test(ica_original, ica_replica)}")
print(f"MOC Welch's: {perform_welchs_test(moc_original, moc_replica)}")

PLS1-SM Welch's: (-1.9237409672041839, 0.07971028563318129)
ICA Welch's: (0.05300939157896869, 0.9584923771821734)
MOC Welch's: (-0.8848277832178829, 0.3930522287334759)


In [21]:
def perform_wilcoxon_test(original, replica):
    w_stat, p_value = wilcoxon(original, replica)
    return w_stat, p_value

print(f"PLS1-SM Wilcoxon: {perform_wilcoxon_test(pls_original, pls_replica)}")
print(f"ICA Wilcoxon: {perform_wilcoxon_test(ica_original, ica_replica)}")
print(f"MOC Wilcoxon: {perform_wilcoxon_test(moc_original, moc_replica)}")

PLS1-SM Wilcoxon: (3.0, 0.0390625)
ICA Wilcoxon: (15.0, 0.7421875)
MOC Wilcoxon: (7.0, 0.1484375)
